In [1]:
# Import section
import cv2
import numpy as np
import requests
import time
import json
import pandas as pd
import sqlalchemy as sa
import pyodbc
import time
from paralel_send import multi_send
from os import getcwd

import argparse
import general_utils


In [4]:
# Info about the params file
p = 'params/campaigns'
camp_file = general_utils.get_conf_file_of_camp(p, str(4))
print("[INFO] Cargando archivo de parámetros: {}".format(camp_file))
dni_area = general_utils.get_param_from_file(camp_file, 'dni_search_area')
traslation_dict = general_utils.get_param_from_file(camp_file, 'traslation_dict')
lines_dict = general_utils.get_param_from_file(camp_file, 'lines_dict')

# Folder params
WORK_DIRECTORY = getcwd()
ARRAYS_PATH = WORK_DIRECTORY + '\\areas_result\\'
TMP_PATH = WORK_DIRECTORY + '\\temps\\'
LOCAL_PATH = WORK_DIRECTORY + '\\local_result\\'
# Execution params
ID_CAMPANIA = 4
ID_USUARIO = 1
# BD params
BD_SAVE_FLAG = True
BD_USERNAME = 'usercupon'
BD_PASSWORD = '123456789'
# BD_DATABASE_NAME = 'ClienteCupon'
BD_DATABASE_NAME = 'DevClienteCupon'
BD_HOST = '192.168.2.55'

eqs = {
    'Normal': 90.00,
    'Low': 60.00
}

banned_field_dict = ['DNI:','DNI','DN']+['Telefono:','Telefono']+['E-mail:']+['Nombres y Apellidos:']+['Direccion:']+['Distrito:']+['Centro Comercial:']
n_eqs = {
    '%': '90',
    ')': '1',
    ',': '',
    'S': '5',
    '$':'5','f':'7','&':'6','+':'7','g':'9','l':'1','o':'0','O':'0','y':'4','/':'1','.':'','-':''
    }

[INFO] Cargando archivo de parámetros: params/marzo_compras


In [16]:
def center_of(bounding_box):
    xs = np.array([bounding_box[0],bounding_box[2],bounding_box[4],bounding_box[6]])
    ys = np.array([bounding_box[1],bounding_box[3],bounding_box[5],bounding_box[7]])
    return int(xs.sum() / xs.size) , int(ys.sum() / ys.size)


def dni_rect_from_ce(x, y, cv2_v, img, areas_dict, lines_dict):
    w = img.shape[1]
    h = img.shape[0]

    color = (255, 0, 0)
    thicknes = 2
    fields = []
    res_dict = {}

    for key in areas_dict.keys():
        tras_vector = np.array([areas_dict[key]['tras_x'], areas_dict[key]['tras_y']])
        point = np.array([x, y]) + tras_vector
        point_2 = point + np.array([areas_dict[key]['width'], areas_dict[key]['height']])

        point = np.where(point<=0,0,point)
        point_2[0] = w if point_2[0]>= w else point_2[0]
        point_2[1] = h if point_2[1]>= h else point_2[1]

        cv2_v.rectangle(img,(point[0],point[1]),(point_2[0],point_2[1]),color,thicknes)
        res_dict[key] = (point, point_2)
        # print('{} {}'.format(point,point_2))

    for key in lines_dict.keys():
        tras_vector = np.array([lines_dict[key]['tras_x'], lines_dict[key]['tras_y']])
        point = np.array([x, y]) + tras_vector
        point_2 = point + np.array([lines_dict[key]['width'], lines_dict[key]['height']])

        point = np.where(point<=0,0,point)
        point_2[0] = w if point_2[0]>= w else point_2[0]
        point_2[1] = h if point_2[1]>= h else point_2[1]

        cv2_v.rectangle(img,(point[0],point[1]),(point_2[0],point_2[1]),color,thicknes)
        res_dict[key] = (point, point_2)

    # return res_dict['dni'][0], res_dict['dni'][1], res_dict['tel'][0], res_dict['tel'][1]
    return res_dict


def is_in_box(center, top_left_point, bottom_right_point):
    x = center[0] > top_left_point[0] and center[0] < bottom_right_point[0]
    y = center[1] > top_left_point[1] and center[1] < bottom_right_point[1]
    return (x and y)


def get_fields_from_json(azure_json, areas_dict):
    fields = {}
    scores = {}
    j=0

    # Setting empty arrays in result
    for key in areas_dict.keys():
        fields[key] = []
        scores[key] = []

    for line in azure_json['recognitionResult']['lines']:
        j += 1
        # print('Line {}: {},{}'.format(j,line['text'],line['boundingBox']))

        # Drawing areas
        cv2.rectangle(image,(line['boundingBox'][0],line['boundingBox'][1]),(line['boundingBox'][2],line['boundingBox'][5]),(0,0,0),1)
        cv2.circle(image,center_of(line['boundingBox']), 4, (0,255,255), -1)

        # Searching in areas_dict
        c = center_of(line['boundingBox'])
        t = line['text']
        ws = []
        scos = []
        for w in line['words']:
            ws.append(w['text'])
            try:
                scos.append(w['confidence'])
            except:
                scos.append('Normal')

        for key in areas_dict.keys():
            # print('C {} está en {}?'.format(c, areas_dict[key]))
            if is_in_box(c, areas_dict[key][0], areas_dict[key][1]):
                # print('String: {} --> {}'.format(t,key))
                fields[key].append(ws)
                scores[key].append(scos)

    return fields, scores


def basic_digit_clean(string, num_eqs):
    for k in num_eqs.keys():
        string = string.replace(k, num_eqs[k])
    return string


def basic_field_clean(string):
    a = string.find(':')
    if a == len(string)-1:
        return string[:-1]
    else:
        if a >= 0:
            return string[a + 1:]
        return string


def post_num_field(l_field, l_score, banned, num_eqs={}, separator=""):
    value = ""
    score = 0.0
    ct = 0

    for i in range(len(l_field)):
        for j in range(len(l_field[i])):
            # print(j)
            if l_field[i][j] not in banned:
                ct += 1
                value += separator + l_field[i][j]
                # print('{}'.format(l_score[i][j]))
                score += eqs[l_score[i][j]]
    value = basic_digit_clean(value, num_eqs)
    return (value.strip(), score / ct) if ct > 0 else (value, 0.0)


def get_dni_from_line(string, num_eqs):
    a = string.find('DNI:')
    b = string.find('Telefono:')
    if a >= 0 and b >= 0:
        return basic_digit_clean(string[a+len('DNI:'):b],num_eqs)
    a = string.find('DNI')
    b = string.find('Telefono')
    if a >= 0 and b >= 0:
        return basic_digit_clean(string[a+len('DNI'):b],num_eqs)
    else:
        return ""


def get_tel_from_line(string, num_eqs):
    b = string.find('Telefono:')
    if b >= 0:
        return basic_digit_clean(string[b+len('Telefono:'):],num_eqs)
    b = string.find('Telefono')
    if b >= 0:
        return basic_digit_clean(string[b+len('Telefono'):],num_eqs)
    else:
        return ""


def clean_letters(word):
    ret = ""
    for c in word:
        if '0' <= c and c <= '9':
            ret += str(c)
    return ret


def has_letters(word):
    ret = True
    for c in word:
        if not '0' <= c and c <= '9':
            return not False
    return not True

In [6]:
# Getting arrays
m_widths = np.genfromtxt(ARRAYS_PATH + 'widths.txt')
m_heights = np.genfromtxt(ARRAYS_PATH + 'heights.txt')
m_cxs = np.genfromtxt(ARRAYS_PATH + 'cxs.txt')
m_cys = np.genfromtxt(ARRAYS_PATH + 'cys.txt')
m_files = np.load(ARRAYS_PATH + 'paths.npy')
m_azure_flags = np.load(ARRAYS_PATH + 'azure_flags.npy')

# Loading muestra numpy arrays
areas_r = pd.read_csv(ARRAYS_PATH + 'areas.csv')
areas_r = areas_r.fillna('')
print('Total cupones area: {}'.format(areas_r.path.values.size))

Total cupones area: 3157


In [8]:
files = []
jsons = []
for i in range(10):
    file = np.load(TMP_PATH + 'n_files_{}.npy'.format(i), allow_pickle=True)
    azu = np.load(TMP_PATH + 'n_jsons_{}.npy'.format(i), allow_pickle=True)
    if file.size > 0 and azu.size > 0:
        files.append(file)
        jsons.append(azu)

azure_files = np.concatenate(files[:], axis=0)
azure_jsons = np.concatenate(jsons[:], axis=0)
print('Resultado de azure: {}  {}'.format(azure_files.size, azure_jsons.size))

Resultado de azure: 3157  3157


In [9]:
# Merging the areas result with the azure result
proc = pd.DataFrame({'path': azure_files, 'azure_json': azure_jsons})
proc = pd.merge(areas_r, proc, how='left', on='path')
print('Total cupones merge: {}'.format(proc.path.values.size))

Total cupones merge: 3157


## Debug

In [17]:
ruta = 'C:\\git\\cuponesWong\\CuponesWong\\notebooks_flow\\input\\'
# idx = np.argwhere(filenames=='MarzoCompras_XM_B5_20190710103950_00558.jpg')[0][0]
idx = 1

# for i in range(azure_jsons.size):
for i in range(idx,idx+1):
    # Getting json info
    j=0
    # Reading the image
    # name = azure_files[i].split('\\')[-1]
    filename = proc.path.values[i]
    print('Para: {}'.format(filename))
    image = cv2.imread(filename)
    
    c_x = int(proc.cx.values[i])
    c_y = int(proc.cy.values[i])
    cupon_areas = dni_rect_from_ce(c_x, c_y+120, cv2, image, traslation_dict, lines_dict)
    
    for line in proc.azure_json.values[i]['recognitionResult']['lines']:
        # final_str += line['text']
        j+=1
        # print('Line {}: {},{}'.format(j,line['text'],line['boundingBox']))
        print('Line {}:'.format(line['text']))
        for word in line['words']:
            print('{} {}'.format(word['text'], word['boundingBox']))
            cv2.rectangle(image,(word['boundingBox'][0],word['boundingBox'][1]),(word['boundingBox'][2],word['boundingBox'][5]),(0,0,255),1)
            cv2.circle(image,center_of(word['boundingBox']), 4, (255,0,255), -1)
            cv2.putText(image,'{}'.format(word['text']), (word['boundingBox'][0],word['boundingBox'][1]), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 0, 0), 1)
        # print('{}'.format(line))
        cv2.rectangle(image,(line['boundingBox'][0],line['boundingBox'][1]),(line['boundingBox'][2],line['boundingBox'][5]),(0,0,0),1)
        cv2.circle(image,center_of(line['boundingBox']), 4, (0,255,255), -1)
    
    
    
    cv2.imshow('image',image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Para: C:\git\cuponesWong\CuponesWong\notebooks_flow\input\MarzoCompras-0303072019121007_00010.jpg
Line Nombres y Apellidos::
Nombres [34, 41, 150, 39, 149, 75, 34, 70]
y [155, 39, 173, 39, 173, 75, 155, 75]
Apellidos: [179, 39, 302, 39, 302, 73, 178, 75]
Line JUAN CARLOS:
JUAN [339, 14, 466, 18, 465, 61, 338, 64]
CARLOS [476, 18, 653, 21, 652, 59, 475, 61]
Line SAlives:
SAlives [114, 87, 312, 77, 313, 121, 115, 131]
Line AnDYA:
AnDYA [429, 90, 591, 84, 590, 122, 428, 127]
Line DNI: 09883016:
DNI: [33, 154, 101, 146, 102, 186, 34, 188]
09883016 [108, 145, 341, 135, 343, 175, 109, 186]
Line Telefono: 94 64 9 8 591:
Telefono: [428, 160, 547, 147, 547, 188, 428, 191]
94 [553, 147, 593, 144, 593, 186, 553, 187]
64 [599, 144, 642, 142, 643, 185, 599, 186]
9 [648, 142, 668, 142, 669, 185, 649, 185]
8 [680, 141, 700, 141, 701, 184, 681, 184]
591 [716, 141, 790, 143, 790, 182, 717, 183]
Line Direccion::
Direccion: [32, 215, 162, 215, 162, 245, 32, 246]
Line AU. PERU:
AU. [210, 203, 300, 203, 29

## Process

In [20]:
# Processing
y_min = dni_area['y_min']
y_max = dni_area['y_max']
x_min = dni_area['x_min']
x_max = dni_area['x_max']

filenames = []

dnis = []
tels = []
mails = []
nombres = []
dirs = []
distris = []
ccs = []

sco_dnis = []
sco_tels = []
sco_mails = []
sco_nombres = []
sco_dirs = []
sco_distris = []
sco_ccs = []

azure_str_jsons = []

# for i in range(azure_files.size):
for i in range(proc.path.values.size):
    # Grabbing the file
    # filename = m_files[i]
    # filename = azure_files[i]
    filename = proc.path.values[i]
    name = filename.split('\\')
    name = name[len(name) - 1]
    name = name.split('.')[0]

    c_x = int(proc.cx.values[i])
    c_y = int(proc.cy.values[i])

    wi = proc.width.values[i]
    hi = proc.height.values[i]

    x_minimo = int(c_x - (wi / 2))
    x_maximo = int(c_x + (wi / 2))
    y_minimo = int(c_y - (hi / 2))
    y_maximo = int(c_y + (hi / 2))

    image = cv2.imread(filename)
    filenames.append(name)

    if (c_x == 9999 or c_y == 9999):
        dnis.append("")
        tels.append("")
        mails.append("")
        nombres.append("")
        dirs.append("")
        distris.append("")
        ccs.append("")

        sco_dnis.append(0.0)
        sco_tels.append(0.0)
        sco_mails.append(0.0)
        sco_nombres.append(0.0)
        sco_dirs.append(0.0)
        sco_distris.append(0.0)
        sco_ccs.append(0.0)

        azure_str_jsons.append("")
    else:
        print('{} . {}'.format(i, name))

        cupon_areas = dni_rect_from_ce(c_x, c_y+y_min, cv2, image, traslation_dict, lines_dict)
        # print('{}'.format(cupon_areas))
        # fields, scores = get_fields_from_json(azure_jsons[i], cupon_areas)
        fields, scores = get_fields_from_json(proc.azure_json.values[i], cupon_areas)

        # Parsing numeric fields
        dni, sdni = post_num_field(fields['dni'],scores['dni'],banned_field_dict,n_eqs)
        tel, stel = post_num_field(fields['tel'],scores['tel'],banned_field_dict,n_eqs)
        dni, tel = basic_field_clean(dni).strip(), basic_field_clean(tel).strip()

        # Parsing mail
        mail, smail = post_num_field(fields['mail'],scores['mail'],banned_field_dict)
        mail = basic_field_clean(mail).lower().strip()

        # Parsing other fields
        nomb1, snomb1 = post_num_field(fields['nomb_ape1'],scores['nomb_ape1'],banned_field_dict,separator=" ")
        nomb2, snomb2 = post_num_field(fields['nomb_ape2'],scores['nomb_ape2'],banned_field_dict,separator=" ")
        dire1, sdire1 = post_num_field(fields['dir1'],scores['dir1'],banned_field_dict,separator=" ")
        dire2, sdire2 = post_num_field(fields['dir2'],scores['dir2'],banned_field_dict,separator=" ")
        dist, sdist = post_num_field(fields['distrito'],scores['distrito'],banned_field_dict,separator=" ")
        cc, scc = post_num_field(fields['cc'],scores['cc'],banned_field_dict,separator=" ")

        nomb1 = basic_field_clean(nomb1).strip()
        nomb2 = basic_field_clean(nomb2).strip()
        dire1 = basic_field_clean(dire1).strip()
        dire2 = basic_field_clean(dire2).strip()
        dist = basic_field_clean(dist).strip()
        cc = basic_field_clean(cc).strip()

        # Concatenating two lines fields
        nomb_ape, snomb_ape = nomb1+" "+nomb2, (snomb1+snomb2)/2
        direccion, sdireccion = dire1+" "+dire2, (sdire1+sdire2)/2

        # Adjusting mail - statitiscaly void mail and azure detection voi 93% success
        if smail == 0 and mail=="":
            smail = 71.00
        
        # Last trying to get DNI
        if sdni==0:
            # print('{}'.format(post_num_field(fields['dni_tel'],scores['dni_tel'],{},separator="")))
            st, sc = post_num_field(fields['dni_tel'],scores['dni_tel'],{},separator=" ")
            dni = get_dni_from_line(st, n_eqs)
            sdni = 0.0 if dni=="" else sc
        # Last trying to get Tel
        if stel==0:
            # print('{}'.format(post_num_field(fields['dni_tel'],scores['dni_tel'],{},separator="")))
            st, sc = post_num_field(fields['dni_tel'],scores['dni_tel'],{},separator=" ")
            tel = get_tel_from_line(st, n_eqs)
            stel = 0.0 if tel=="" else sc

        dni = dni.replace(" ","")
        tel = tel.replace(" ","")
        # Showing fields
        # print('DNI {} {}'.format(dni, sdni))
        # print('TEL {} {}'.format(tel, stel))
        # print('MAIL {} {}'.format(mail, smail))
        # print('NOMB {} {}'.format(nomb_ape, snomb_ape))
        # print('NOMB2 {} {}'.format(nomb2, snomb2))
        # print('DIR {} {}'.format(direccion, sdireccion))
        # print('DIR2 {} {}'.format(dire2, sdire2))
        # print('DIST {} {}'.format(dist, sdist))
        # print('CC {} {}'.format(cc, scc))

        # Appending into arrays
        dnis.append(dni)
        tels.append(tel)
        mails.append(mail)
        nombres.append(nomb_ape)
        dirs.append(direccion)
        distris.append(dist)
        ccs.append(cc)

        sco_dnis.append(sdni)
        sco_tels.append(stel)
        sco_mails.append(smail)
        sco_nombres.append(snomb_ape)
        sco_dirs.append(sdireccion)
        sco_distris.append(sdist)
        sco_ccs.append(scc)

        azure_str_jsons.append(json.dumps(proc.azure_json.values[i]))

print('Termino :D ')

0 . MarzoCompras-0303072019121007_00008
1 . MarzoCompras-0303072019121007_00010
2 . MarzoCompras-0303072019121007_00025
3 . MarzoCompras-0303072019121013_00265
4 . MarzoCompras-0303072019121014_00331
5 . MarzoCompras-0303072019121014_00343
6 . MarzoCompras-0303072019121017_00450
7 . MarzoCompras-0303072019121017_00459
8 . MarzoCompras-0303072019121018_00497
9 . MarzoCompras-0303072019121018_00501
10 . MarzoCompras-0303072019121019_00514
11 . MarzoCompras-0303072019121019_00540
12 . MarzoCompras-0303072019121019_00541
13 . MarzoCompras-0303072019121021_00570
14 . MarzoCompras-0303072019121021_00585
15 . MarzoCompras-0303072019121023_00648
16 . MarzoCompras-0303072019121023_00651
17 . MarzoCompras-0303072019121023_00652
18 . MarzoCompras-0303072019121024_00709
19 . MarzoCompras-0303072019121025_00771
20 . MarzoCompras-0303072019121030_00931
21 . MarzoCompras-0303072019121032_00998
22 . MarzoCompras-0303072019125009_00002
23 . MarzoCompras-0303072019125011_00105
24 . MarzoCompras-03030720

216 . MarzoCompras_AV_B3_20190705174248_00667
217 . MarzoCompras_AV_B3_20190705174252_00762
218 . MarzoCompras_AV_B3_20190705174256_00916
219 . MarzoCompras_AV_B3_20190705174257_00961
220 . MarzoCompras_AV_B3_20190705180843_00020
221 . MarzoCompras_AV_B3_20190705180848_00193
222 . MarzoCompras_AV_B3_20190705180848_00227
223 . MarzoCompras_AV_B3_20190705180855_00466
224 . MarzoCompras_AV_B3_20190705180900_00668
225 . MarzoCompras_AV_B3_20190705180901_00706
226 . MarzoCompras_AV_B3_20190705180903_00751
227 . MarzoCompras_AV_B3_20190705180904_00821
228 . MarzoCompras_AV_B3_20190705180908_00946
229 . MarzoCompras_AV_B3_20190708093415_00068
230 . MarzoCompras_AV_B3_20190708093416_00109
231 . MarzoCompras_AV_B3_20190708093418_00171
232 . MarzoCompras_AV_B3_20190708093419_00230
233 . MarzoCompras_AV_B3_20190708093421_00293
234 . MarzoCompras_AV_B3_20190708093421_00299
235 . MarzoCompras_AV_B3_20190708093424_00438
236 . MarzoCompras_AV_B3_20190708093424_00443
237 . MarzoCompras_AV_B3_201907080

411 . MarzoCompras_AV_B5_20190709155402_00108
412 . MarzoCompras_AV_B5_20190709155406_00267
413 . MarzoCompras_AV_B5_20190709155407_00301
414 . MarzoCompras_AV_B5_20190709155409_00384
415 . MarzoCompras_AV_B5_20190709155410_00402
416 . MarzoCompras_AV_B5_20190709155410_00404
417 . MarzoCompras_AV_B5_20190709155410_00409
418 . MarzoCompras_AV_B5_20190709155412_00485
419 . MarzoCompras_AV_B5_20190709155414_00550
420 . MarzoCompras_AV_B5_20190709155419_00729
421 . MarzoCompras_AV_B5_20190709155420_00771
422 . MarzoCompras_AV_B5_20190709155421_00781
423 . MarzoCompras_AV_B5_20190709155421_00786
424 . MarzoCompras_AV_B5_20190709155421_00793
425 . MarzoCompras_AV_B5_20190709155421_00796
426 . MarzoCompras_AV_B5_20190709155423_00867
427 . MarzoCompras_AV_B5_20190709155424_00890
428 . MarzoCompras_AV_B5_20190709155425_00941
429 . MarzoCompras_AV_B5_20190709155426_00971
430 . MarzoCompras_AV_B5_20190709155426_00981
431 . MarzoCompras_AV_B5_20190709155427_01000
432 . MarzoCompras_AV_B5_201907091

598 . MarzoCompras_EL_B2_20190703180756_00954
599 . MarzoCompras_EL_B2_20190704103502_00007
600 . MarzoCompras_EL_B2_20190704103503_00042
601 . MarzoCompras_EL_B2_20190704103507_00174
602 . MarzoCompras_EL_B2_20190704103510_00293
603 . MarzoCompras_EL_B2_20190704103511_00322
604 . MarzoCompras_EL_B2_20190704103516_00490
605 . MarzoCompras_EL_B2_20190704103516_00518
606 . MarzoCompras_EL_B2_20190704103517_00523
607 . MarzoCompras_EL_B2_20190704103518_00576
608 . MarzoCompras_EL_B2_20190704103518_00590
609 . MarzoCompras_EL_B2_20190704103519_00608
610 . MarzoCompras_EL_B2_20190704103520_00668
611 . MarzoCompras_EL_B2_20190704103521_00702
612 . MarzoCompras_EL_B2_20190704103523_00765
613 . MarzoCompras_EL_B2_20190704103526_00857
614 . MarzoCompras_EL_B2_20190704103526_00860
615 . MarzoCompras_EL_B2_20190704114113_00011
616 . MarzoCompras_EL_B2_20190704114115_00074
617 . MarzoCompras_EL_B2_20190704114115_00084
618 . MarzoCompras_EL_B2_20190704114115_00088
619 . MarzoCompras_EL_B2_201907041

784 . MarzoCompras_EL_B2_20190705120539_00554
785 . MarzoCompras_EL_B2_20190705120540_00566
786 . MarzoCompras_EL_B2_20190705120540_00594
787 . MarzoCompras_EL_B2_20190705120541_00613
788 . MarzoCompras_EL_B2_20190705120542_00638
789 . MarzoCompras_EL_B2_20190705120542_00649
790 . MarzoCompras_EL_B2_20190705120543_00664
791 . MarzoCompras_EL_B2_20190705120546_00767
792 . MarzoCompras_EL_B2_20190705120548_00829
793 . MarzoCompras_EL_B2_20190705120548_00845
794 . MarzoCompras_EL_B2_20190705120549_00869
795 . MarzoCompras_EL_B2_20190705120550_00887
796 . MarzoCompras_EL_B2_20190705120550_00910
797 . MarzoCompras_EL_B2_20190705130035_00006
798 . MarzoCompras_EL_B2_20190705130035_00007
799 . MarzoCompras_EL_B2_20190705130035_00017
800 . MarzoCompras_EL_B2_20190705130037_00062
801 . MarzoCompras_EL_B2_20190705130037_00069
802 . MarzoCompras_EL_B2_20190705130038_00108
803 . MarzoCompras_EL_B2_20190705130043_00273
804 . MarzoCompras_EL_B2_20190705130043_00280
805 . MarzoCompras_EL_B2_201907051

...

2315 . MarzoCompras_MD_B2_20190705134830_00151
2316 . MarzoCompras_MD_B2_20190705134831_00183
2317 . MarzoCompras_MD_B2_20190705134834_00277
2318 . MarzoCompras_MD_B2_20190705134838_00462
2319 . MarzoCompras_MD_B2_20190705134840_00529
2320 . MarzoCompras_MD_B2_20190705134840_00533
2321 . MarzoCompras_MD_B2_20190705134840_00535
2322 . MarzoCompras_MD_B2_20190705134841_00572
2323 . MarzoCompras_MD_B2_20190705134842_00606
2324 . MarzoCompras_MD_B2_20190705134846_00684
2325 . MarzoCompras_MD_B2_20190705134847_00715
2326 . MarzoCompras_MD_B2_20190705134850_00784
2327 . MarzoCompras_MD_B2_20190705134850_00805
2328 . MarzoCompras_MD_B2_20190705134851_00833
2329 . MarzoCompras_MD_B2_20190705134853_00903
2330 . MarzoCompras_MD_B2_20190705134854_00924
2331 . MarzoCompras_MD_B2_20190705134855_00948
2332 . MarzoCompras_MD_B2_20190705140403_00034
2333 . MarzoCompras_MD_B2_20190705140404_00042
2334 . MarzoCompras_MD_B2_20190705140404_00063
2335 . MarzoCompras_MD_B2_20190705140406_00120
2336 . MarzoC

2493 . MarzoCompras_MD_B4_20190708181055_00650
2494 . MarzoCompras_MD_B4_20190708181058_00785
2495 . MarzoCompras_MD_B4_20190708181058_00793
2496 . MarzoCompras_MD_B4_20190708181058_00794
2497 . MarzoCompras_MD_B4_20190708181100_00830
2498 . MarzoCompras_MD_B4_20190708181101_00843
2499 . MarzoCompras_MD_B4_20190708181102_00896
2500 . MarzoCompras_MD_B4_20190708181102_00904
2501 . MarzoCompras_MD_B4_20190708181103_00940
2502 . MarzoCompras_MD_B4_20190708181104_00951
2503 . MarzoCompras_MD_B4_20190709092552_00018
2504 . MarzoCompras_MD_B4_20190709092554_00075
2505 . MarzoCompras_MD_B4_20190709092554_00092
2506 . MarzoCompras_MD_B4_20190709092557_00177
2507 . MarzoCompras_MD_B4_20190709092558_00196
2508 . MarzoCompras_MD_B4_20190709092600_00266
2509 . MarzoCompras_MD_B4_20190709092603_00386
2510 . MarzoCompras_MD_B4_20190709092605_00472
2511 . MarzoCompras_MD_B4_20190709092606_00496
2512 . MarzoCompras_MD_B4_20190709092608_00567
2513 . MarzoCompras_MD_B4_20190709092610_00621
2514 . MarzoC

2682 . MarzoCompras_MD_B5_20190710104622_00086
2683 . MarzoCompras_MD_B5_20190710104625_00219
2684 . MarzoCompras_MD_B5_20190710104626_00247
2685 . MarzoCompras_MD_B5_20190710104629_00390
2686 . MarzoCompras_MD_B5_20190710104630_00435
2687 . MarzoCompras_MD_B5_20190710104636_00678
2688 . MarzoCompras_MD_B5_20190710104637_00711
2689 . MarzoCompras_MD_B5_20190710104638_00754
2690 . MarzoCompras_MD_B5_20190710104638_00755
2691 . MarzoCompras_MD_B5_20190710104639_00782
2692 . MarzoCompras_MD_B5_20190710104639_00798
2693 . MarzoCompras_MD_B5_20190710104640_00841
2694 . MarzoCompras_MD_B5_20190710104641_00869
2695 . MarzoCompras_MD_B5_20190710104643_00944
2696 . MarzoCompras_MD_B5_20190710104644_00982
2697 . MarzoCompras_MD_B5_20190710112237_00030
2698 . MarzoCompras_MD_B5_20190710112237_00034
2699 . MarzoCompras_MD_B5_20190710112240_00133
2700 . MarzoCompras_MD_B5_20190710112240_00143
2701 . MarzoCompras_MD_B5_20190710112240_00153
2702 . MarzoCompras_MD_B5_20190710112241_00175
2703 . MarzoC

2880 . MarzoCompras_XM_B4_20190708165936_00902
2881 . MarzoCompras_XM_B4_20190708165938_00948
2882 . MarzoCompras_XM_B4_20190708165939_00985
2883 . MarzoCompras_XM_B4_20190708165939_00993
2884 . MarzoCompras_XM_B4_20190709095432_00051
2885 . MarzoCompras_XM_B4_20190709095435_00062
2886 . MarzoCompras_XM_B4_20190709095451_00132
2887 . MarzoCompras_XM_B4_20190709095500_00170
2888 . MarzoCompras_XM_B4_20190709095501_00174
2889 . MarzoCompras_XM_B4_20190709095514_00230
2890 . MarzoCompras_XM_B4_20190709095531_00306
2891 . MarzoCompras_XM_B4_20190709095544_00344
2892 . MarzoCompras_XM_B4_20190709095551_00377
2893 . MarzoCompras_XM_B4_20190709095603_00439
2894 . MarzoCompras_XM_B4_20190709095613_00480
2895 . MarzoCompras_XM_B4_20190709095628_00527
2896 . MarzoCompras_XM_B4_20190709095648_00635
2897 . MarzoCompras_XM_B4_20190709095656_00778
2898 . MarzoCompras_XM_B4_20190709095702_00851
2899 . MarzoCompras_XM_B4_20190709095705_00903
2900 . MarzoCompras_XM_B4_20190709095724_00986
2901 . MarzoC

3070 . MarzoCompras_XM_B5_20190710113232_00887
3071 . MarzoCompras_XM_B5_20190710113233_00892
3072 . MarzoCompras_XM_B5_20190710113235_00929
3073 . MarzoCompras_XM_B5_20190710113238_00979
3074 . MarzoCompras_XM_B5_20190710113238_00993
3075 . MarzoCompras_XM_B5_20190710113239_01031
3076 . MarzoCompras_XM_B5_20190710113240_01059
3077 . MarzoCompras_XM_B5_20190710113243_01121
3078 . MarzoCompras_XM_B5_20190710113243_01134
3079 . MarzoCompras_XM_B5_20190710122814_00048
3080 . MarzoCompras_XM_B5_20190710122814_00064
3081 . MarzoCompras_XM_B5_20190710122815_00081
3082 . MarzoCompras_XM_B5_20190710122818_00183
3083 . MarzoCompras_XM_B5_20190710122818_00184
3084 . MarzoCompras_XM_B5_20190710122819_00207
3085 . MarzoCompras_XM_B5_20190710122820_00235
3086 . MarzoCompras_XM_B5_20190710122821_00274
3087 . MarzoCompras_XM_B5_20190710122825_00406
3088 . MarzoCompras_XM_B5_20190710122827_00425
3089 . MarzoCompras_XM_B5_20190710122827_00458
3090 . MarzoCompras_XM_B5_20190710122832_00524
3091 . MarzoC

In [21]:
bd_azure = pd.DataFrame({'NombreArchivo': np.array(filenames)})
# bd_azure['idCupon'] = np.arange(1000, 1000+np.array(dnis).size,1)
bd_azure['DNI'] = np.array(dnis)
bd_azure['DNI'] = np.where(bd_azure['DNI'].apply(len) >= 14,
    bd_azure['DNI'].apply(lambda x : clean_letters(x)[:8]),
    bd_azure['DNI']
    )

bd_azure['AcertividadDNI'] = np.array(sco_dnis)
bd_azure['AcertividadDNI'] = np.where(np.logical_and(bd_azure['DNI'].apply(len) <= 9,
                                                     np.logical_and(bd_azure['DNI'].apply(len) >= 8,
                                                                    np.logical_not(bd_azure['DNI'].apply(has_letters))
                                                                    )
                                                     ),
                                      bd_azure['AcertividadDNI'],
                                      0.0
                                      )

bd_azure['Telefono'] = np.array(tels)
bd_azure['AcertividadTelefono'] = np.array(sco_tels)
bd_azure['AcertividadTelefono'] = np.where(np.logical_and(np.logical_or(bd_azure['Telefono'].apply(len) == 9,
                                                                   np.logical_or(bd_azure['Telefono'].apply(len) == 7,
                                                                                 bd_azure['Telefono'].apply(len) == 6)
                                                                   ),
                                                     np.logical_not(bd_azure['Telefono'].apply(has_letters))
                                                     ),
                                           bd_azure['AcertividadTelefono'],
                                           0.0
                                           )

bd_azure['NombreCompleto'] = np.array(nombres)
bd_azure['AcertividadNombreCompleto'] = np.array(sco_nombres)

bd_azure['Direccion'] = np.array(dirs)
bd_azure['AcertividadDireccion'] = np.array(sco_dirs)

bd_azure['Distrito'] = np.array(distris)
bd_azure['AcertividadDistrito'] = np.array(sco_distris)

bd_azure['Correo'] = np.array(mails)
# bd_azure['Correo'] = bd_azure['Correo'].apply(str.lower)

bd_azure['AcertividadCorreo'] = np.array(sco_mails)

bd_azure['AzureJsonOCR'] = np.array(azure_str_jsons)

bd_azure['idCampania'] = ID_CAMPANIA
bd_azure['idUsuario'] = ID_USUARIO
bd_azure['idEstado'] = 2
# bd_azure['idBatch'] = int(dt_string)

bd_azure.to_csv('azure_result/azure_result_testing.csv', encoding='utf-8', index=False)
print('Termino escritura')

Termino escritura
